# 第二章

In [ ]:
# 2.2 从MySql数据库中读取交易流水数据
import pymysql

# 建立连接对象
connection = pymysql.connect(host = '172.17.6.26',
                    port = 3306,
                   user = 'raa_user',
                   password = 'bigdata123',
                    db = 'transactions',
                    charset = 'utf8mb4')

try:
    with connection.cursor() as cursor:
        # 查询数据表行数
        sql1 = "SELECT count(*) FROM flow_data"
        # 选取表的前五行
        sql2 = "SELECT * FROM flow_data LIMIT 5"

        cursor.execute(sql1)
        rows = cursor.fetchall() 
        cursor.execute(sql2)
        head = cursor.fetchall() 

finally:
    connection.close()

# 查看数据的行列数
print('数据行数为:',rows,'\n前五行数据为:',head)

In [ ]:
# 2.3 对客户数据进行格式转换
import pandas as pd

# 将data转换为DataFrame格式
data = pd.DataFrame(list(data),columns=['user_id','payment','describe','unix_time'])

# 第三章

In [1]:
# 3.2 数据统计分析

import pandas as pd

# 查看数据形状
rows, cols = data.shape

print("数据共有 %s 行，有 %s 列。" % (rows, cols))

# 查看数据的前五行
head = data.head()

print('\n',head)

# 查看数据的基本情况
data.info(null_counts = True)

NameError: name 'data' is not defined

In [ ]:
# 3.3 查看客户总数
import pandas as pd

# 计算客户个数
user_num = len(set(data['user_id']))
print("客户总数为:",user_num)

# 计算客户交易次数
user_counts = data['user_id'].value_counts()
print("每个客户的交易次数为:\n",user_counts)

In [ ]:
# 3.4 交易时间异常值检测
import pandas as pd

# 书写正则表达式
pattern = '^\d{10}$'

# 筛选异常值
outlier = data[~data['unix_time'].str.match(pattern)]  # 不是10位数字组成的

# 统计不同类型的异常值及其数量
outlier_counts = outlier['unix_time'].value_counts()

print(outlier_counts)


In [ ]:
# 3.5 交易时间异常值处理
import pandas as pd

# 去掉交易时间为0的行
data = data.loc[data['unix_time'] != 0]

# 将异常值填补为正常值
data.loc[data['unix_time'] == '14 3264000','unix_time'] = 1473264000

print(data.loc[data['unix_time'] == '14 3264000'])
print(data.loc[data['unix_time'] == 0])

In [ ]:
# 3.6 交易金额异常值处理
import pandas as pd

# 查看交易金额为'\N'的行数
print("交易金额异常的记录共有%s行。" % (len(data[data['payment'] == '\\N'])))

# 去除交易金额为'\N'的行
data = data[data['payment'] != '\\N']

print(data[data['payment'] == '\\N'])

In [ ]:
# 3.7 交易附言缺失值处理
import pandas as pd

# 筛选describe中有无附言为空的行
describe_null =  data[data['describe'].isnull()]

print("交易附言为空的行共有%s条。"%  len(describe_null))
print(describe_null.head())

In [ ]:
# 3.8 时间格式和时区转换
import pandas as pd

# 时间格式转换
data['pay_time'] = pd.to_datetime(data['unix_time'],unit = 's')

# 时区转换
data['pay_time'] = data['pay_time'] + pd.Timedelta(hours = 8)

print(data.tail(5))

In [ ]:
# 3.9 量纲转换
import pandas as pd

# 将payment列标准化
data['payment'] = data['payment']/100

print(data.head())

In [ ]:
# 3.10重复数据处理
import pandas as pd

# 检测重复值
duplicate_values = data[data.duplicated()]
print("重复数据有%s行。"% len(duplicate_values))

# 去掉重复值
data.drop_duplicates(inplace = True) # 在原数据上进行更改
print("处理之后，交易记录变为%s行。" % len(data))

# 第四章

In [ ]:
# 4.2 交易次数随时间的可视化分析
import pandas as pd
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))

# 绘制折线图
data['pay_time'].dt.date.value_counts().plot()

# 设置图形标题
plt.title('不同时间的交易次数分布')

# 设置y轴标签
plt.ylabel('交易次数')

# 设置x轴标签
plt.xlabel('时间')

plt.show()

In [ ]:
# 4.3 交易金额随时间的可视化分析
import pandas as pd
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))

# 绘制折线图
abs(data['payment']).groupby(data['pay_time'].dt.date).sum().plot()

# 设置图形标题
plt.title('不同时间的交易金额分布')

# 设置y轴标签
plt.ylabel('交易金额')

# 设置x轴标签
plt.xlabel('时间')

plt.show()

In [ ]:
# 4.4 交易有效时间段的限定
import pandas as pd

# 时间限定
data = data[(data['pay_time'] <= pd.Timestamp(2018,1,1)) & (data['pay_time'] >= pd.Timestamp(2016,7,1))]

print(data.shape)

In [ ]:
# 4.5 每天24小时交易次数的分布
import pandas as pd
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,6))

# 绘制条形图
data['pay_time'].dt.hour.value_counts().sort_index().plot.bar(color = 'orange',rot = 360)

# 设置图形标题
plt.title('每天24小时的交易次数分布')

# 设置x轴标签
plt.xlabel('小时')

# 设置y轴标签
plt.ylabel('交易次数')

plt.show()

In [ ]:
# 4.6 客户交易次数的可视化分布
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 5))

# 绘制核密度图
sns.kdeplot(data['user_id'].value_counts(),shade = True,legend = False)

# 设置x，y轴标签
plt.xlabel('交易次数')
plt.ylabel('频率')

# 设置图的标题
plt.title('客户交易次数')

plt.show()

In [ ]:
# 4.7 客户平均交易金额的可视化分析
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 5))

# 绘制核密度图
sns.kdeplot(abs(data['payment']).groupby(data['user_id']).mean(),shade = True,legend = False)

# 设置x，y轴标签
plt.xlabel('平均交易金额')
plt.ylabel('频率')

# 设置图的标题
plt.title('客户平均交易金额的分布')

plt.show()

In [ ]:
# 4.8 客户交易流入流出次数的可视化分析
import seaborn as sns

fig,[ax1,ax2] = plt.subplots(1,2,figsize=(16, 5))

# 定义和选取金额流入流出的交易记录
input_payment = data[data['payment'] < 0]
output_payment = data[data['payment'] > 0]

# 计算每个客户的流入流出次数
input_payment_count = input_payment.groupby('user_id').size()
output_payment_count = output_payment.groupby('user_id').size()

# 绘制直方图
sns.distplot(input_payment_count,ax=ax1)
sns.distplot(output_payment_count,ax=ax2)

# 设置标题
ax1.set_title('客户交易的流入次数分布')
ax2.set_title('客户交易的流出次数分布')

plt.show()

In [ ]:
# 4.9 客户交易流入流出金额的可视化分析
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 定义和选取金额流入流出的交易记录
input_payment = data[data['payment'] < 0]
output_payment = data[data['payment'] > 0]

# 计算每个客户的流入流出金额
input_payment_amount = input_payment.groupby('user_id')['payment'].mean()
output_payment_amount = output_payment.groupby('user_id')['payment'].mean()

fig,[ax1,ax2] = plt.subplots(1,2,figsize=(16, 5))

# 绘制直方图
sns.distplot(input_payment_amount,ax = ax1)
sns.distplot(output_payment_amount,ax = ax2)

# 设置标题
ax1.set_title('客户交易的平均流入金额分布')
ax2.set_title('客户交易的平均流出金额分布')

plt.show()

In [ ]:
# 4.10 交易附言文本预处理
import pandas as pd
import jieba

# 数据采样
data = data.sample(20000,random_state = 22)

# 文本分词
data['describe_cutted'] = data['describe'].apply(lambda x:" ".join(jieba.cut(x)))

# 过滤停用词  
def del_stopwords(words):  
    output = ''  
    for word in words:  
        if word not in stopwords:  
            output += word
    return output

data['describe_cutted'] = data['describe_cutted'].apply(del_stopwords)

print(data.head())


In [ ]:
# 4.11 交易附言词云绘制
from wordcloud import WordCloud         
import matplotlib.pyplot as plt

# 数据采样
data = data.sample(20000,random_state = 22)

# 文本拼接
describe_document = " ".join(data['describe_cutted'])

fig = plt.figure(figsize=(20,10))

# 创建词云对象
wordcloud = WordCloud(background_color = 'white',
                      font_path = "FangSong.ttf",
                      mask = background_Image,
                      collocations = False,
                      scale = 2,
                      random_state = 30)

# 生成词云
wordcloud.generate(describe_document)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# 4.12 交易附言关键词提取
import jieba
       
# 提取关键词
tags = jieba.analyse.extract_tags(describe_document,withWeight = True,topK = 50)

# 输出关键词
for tag in tags:
    print(tag)

# 第五章

In [ ]:
# 5.2 事实类标签的计算（交易次数和交易总额）
import pandas as pd

user_features = pd.DataFrame(index = data["user_id"].unique())

# 计算客户的交易次数
user_features['total_transactions_cnt'] = data.groupby('user_id').size()

# 计算客户的交易总金额
user_features['total_transactions_amt'] = abs(data.payment).groupby(data['user_id']).sum()

print(user_features.head())

In [ ]:
# 5.3 事实类标签的计算（转账次数和转账金额）
import pandas as pd

# 提取包含转账行为的记录
transfer = data[data['describe'].str.contains('转账')] #附言中是否有转账这个词

# 计算客户的转账次数
user_features["transfer_cnt"] = transfer.groupby('user_id').size()

# 计算客户的转账总金额
user_features["transfer_amt"] = abs(transfer.payment).groupby(transfer['user_id']).sum()

# 计算客户的转账平均金额
user_features["transfer_mean"] = abs(transfer.payment).groupby(transfer['user_id']).mean()

print(user_features.head())

In [ ]:
# 5.4 事实类标签的计算（其他）
import pandas as pd

# 1.单次最大消费金额
user_features["max_consume_amt"] = consume.groupby(by='user_id').apply(lambda x: max(x['payment']))

# 2.消费订单比例
user_features["consume_order_ratio"] = consume.groupby(by='user_id').size() / user_features['total_transactions_cnt']

# 3.月均消费频度
standard_day = pd.Timestamp(2018,1,1)
consume['distance'] = (standard_day - consume['pay_time']) / pd.Timedelta(days=30)
user_features['mon_consume_frq'] = consume.groupby(by='user_id').apply(lambda x: len(x) / x['distance'].max())

# 4.最常用支付工具
consume_info = data[data.payment > 0]   
pay = consume_info[consume_info["describe"].str.contains('转账|支付|还款|充值|交易')]  # 支付
pay['paytool'] = ''
for i in range(len(pay)):
   if '支付宝' in pay.iloc[i,2]:
    pay.iloc[i,-1] = 'Alipay'
   elif '淘宝' in pay.iloc[i,2]:
    pay.iloc[i,-1] = 'Alipay'
   elif '余额宝' in pay.iloc[i,2]:
    pay.iloc[i,-1] = 'Alipay'
   elif '信用卡' in pay.iloc[i,2]:
    pay.iloc[i,-1] = 'CreditcardPay'
   elif '银联' in pay.iloc[i,2]:
    pay.iloc[i,-1] = 'UnionPay'
   else:
    pay.iloc[i,-1] = 'OthersPay'
user_features['consumption_channel'] = pay['paytool'].groupby('user_id').value_counts().max()


# 5.网购订单次数
user_features['online_cnt'] = online.groupby(by='user_id').size()

# 6.网购订单总金额
user_features['online_amt'] = online.groupby(by='user_id')['payment'].sum()

# 7.网购订单平均金额
user_features['online_avg_amt'] = online.groupby(by='user_id')['payment'].mean()

# 8.月均网购频度
user_features['mon_online_frq'] = online.groupby(by='user_id').apply(lambda x: len(x) / x['distance'].max())

# 9.网购首单时间
user_features['online_buy_first_date'] = online.groupby(by='user_id').apply(lambda x: pd.Timestamp(2018,1,1) - pd.Timedelta(days=x.max()))

# 10.网购尾单时间
user_features['online_buy_last_date'] = online.groupby(by='user_id').apply(lambda x: pd.Timestamp(2018,1,1) - pd.Timedelta(days=x.min()))

# 提取餐饮消费的记录
dining = consume[consume["describe"].str.contains("餐饮|食")]

# 11.餐饮订单次数
user_features['dining_cnt'] = dining.groupby(by='user_id').size()

# 12.餐饮订单总金额
user_features['dining_amt'] = dining.groupby(by='user_id')['payment'].sum()

# 13.餐饮订单平均金额
user_features['ding_avg_amt'] = dining.groupby(by='user_id')['payment'].mean()

# 提取商旅消费的记录
train_bus = consume[consume["describe"].str.contains("车票|火车票|机票")]
travel_hotel = consume[consume["describe"].str.contains("住宿|宾馆|酒店|携程|去哪")]
business_travel = pd.concat([train_bus,travel_hotel])

# 14.商旅消费次数
user_features["business_travel_cnt"] = business_travel.groupby("user_id").size()

# 15.商旅消费总金额
user_features["business_travel_amt"] = business_travel.payment.groupby(business_travel["user_id"]).sum()

# 16.商旅消费平均金额
user_features["business_travel_avg_amt"] = business_travel.payment.groupby(business_travel["user_id"]).mean()

# 17.月均旅行频次
user_features['mon_business_travel_frq'] = business_travel.groupby(by='user_id').apply(lambda x: len(x) / x['distance'].max())

# 提取汽车消费记录
car = consume[consume['describe'].str.contains('汽车')]

# 18.汽车消费次数
user_features['car_cnt'] = car.groupby(by='user_id').size()

# 19.汽车消费总金额
user_features['car_amt'] = car.groupby(by='user_id')['payment'].sum()

# 20.有无代发
user_features['payroll'] = consume.groupby(by='user_id').apply(lambda x: '代发' in ' '.join(x['describe']))

# 提取退货记录
return_goods = consume_less[consume_less['describe'].str.contains('退货')]

# 21.退货订单数
user_features['return_cnt'] = return_goods.groupby(by='user_id').size()

# 提取公共事业缴费的记录
public = consume_info[consume_info["describe"].str.contains('水费|电费|燃气费')]

# 22.公共事业缴费总额
user_features["public_pay_amt"] = public.groupby("user_id")['payment'].sum()

# 提取网络媒体类消费记录
net_media = consume[consume['describe'].str.contains('充值|会员')]

# 23.网络媒体类消费次数
user_features['internet_media_cnt'] = net_media.groupby(by='user_id').size()

# 24.网络媒体类消费总金额
user_features['internet_media_amt'] = net_media.groupby(by='user_id')['payment'].sum()

# 提取话费消费记录
tele_dill = consume[consume['describe'].str.contains('话费')]

# 25.话费通讯类消费次数
user_features['phone_fee_cnt'] = tele_dill.groupby(by='user_id').size()

# 26.话费通讯类消费总金额
user_features['phone_fee_amt'] = tele_dill.groupby(by='user_id')['payment'].sum()

# 27.有无分期
user_features['is_installment'] = consume.groupby(by='user_id').apply(lambda x: '分期' in ' '.join(x['describe']))

# 提取预借记录
pre_lend = consume_less[consume_less['describe'].str.contains('预借')]

# 28.预借现金次数
user_features['cash_advance_cnt'] = pre_lend.groupby(by='user_id').size()

# 29.预借现金总金额
user_features['cash_advance_amt'] = pre_lend.groupby(by='user_id')['payment'].sum()

# 30.计算客户的交易次数(上面有)
user_features['total_transactions_cnt'] = data.groupby(by='user_id').apply(lambda x: len(x))

# 31.计算客户的交易总金额(上面有)
user_features['total_transactions_amt'] = data.groupby(by='user_id').apply(lambda x: sum(abs(x['payment'])))

# 提取提现记录
withdraw = consume[consume['describe'].str.contains('提现')]

# 32.提现次数
user_features['withdraw_cnt'] = withdraw.groupby(by='user_id').size()

# 33.提现总金额
user_features['withdraw_amt'] = withdraw.groupby(by='user_id')['payment'].sum()

# 34.ATM存款总金额
user_features['total_deposit'] = consume[consume['describe'].str.contains('ATM存款')].groupby(by='user_id')['payment'].sum()

# 35.ATM取款总金额
user_features['total_withdraw'] = consume[consume['describe'].str.contains('ATM取款')].groupby(by='user_id')['payment'].sum()

# 提取转账记录
transfer = data[data['describe'].str.contains('转账')]

# 36.计算客户的转账次数
user_features["transfer_cnt"] = transfer.groupby(by='user_id').apply(lambda x: len(x))

# 37.计算客户的转账总金额
user_features["transfer_amt"] = transfer.groupby(by='user_id').apply(lambda x: sum(abs(x['payment'])))

# 38.计算客户的转账平均金额
user_features["transfer_mean"] = transfer.groupby(by='user_id').apply(lambda x: abs(x['payment']).mean())

# 提取信用卡还款记录
credit = consume_info[consume_info['describe'].str.contains('信用卡还款')]

# 39.信用卡还款次数
user_features['credit_card_repay_cnt'] = credit.groupby(by='user_id').size()

# 40.信用卡还款总金额
user_features['credit_card_repay_amt'] = credit.groupby(by='user_id')['payment'].sum()

In [ ]:
# 5.5 规则类标签的计算（有无高端消费）
import pandas as pd

# 计算阈值
threshold = user_features['max_consume_amt'].quantile(0.75)

# 判定有无高端消费
user_features["high_consumption"] = user_features['max_consume_amt'].apply(lambda x: 1 if x > threshold else 0)

print(user_features.head())

In [ ]:
# 5.6 规则类标签的计算（是否休眠客户）
import pandas as pd

# 计算阈值  下四分位数
threshold = user_features['total_transactions_cnt'].quantile(0.25)

# 判定是否休眠
user_features['sleep_customers'] = user_features['total_transactions_cnt'].apply(lambda x:1 if x < threshold else 0)

print(user_features.head())

In [ ]:
# 5.8 规则类标签的计算（计算时间差）
import pandas as pd
import numpy as np

# 选取观察点
standard_day = pd.Timestamp(2018,1,1)

# 计算天数
consume['distance'] = standard_day - consume['pay_time']
consume['distance'] = consume['distance'] / pd.Timedelta(days = 1)

print(consume.head())

In [ ]:
# 5.9 规则类标签的计算（近度，频度，值度）
import pandas as pd

# 计算rfm
user_features[['recency','frequency','monetary']] = consume.groupby('user_id').agg({'distance':'min','user_id':'size','payment':'sum'})

print(user_features.head())

In [ ]:
# 5.10 规则类标签的计算（RFM总得分）
import pandas as pd

# 等频离散化
user_features['R_score'] = pd.qcut(user_features['recency'],4,labels = [4,3,2,1])
user_features['F_score'] = pd.qcut(user_features['frequency'],4,labels = [1,2,3,4])
user_features['M_score'] = pd.qcut(user_features['monetary'],4,labels = [1,2,3,4])

# 填充缺失值
user_features[['R_score','F_score','M_score']] = user_features[['R_score','F_score','M_score']].fillna(1)

# 计算总得分
user_features['Total_Score'] = user_features.R_score.astype('int') + user_features.F_score.astype('int') + user_features.M_score.astype('int')

print(user_features.head())

In [ ]:
# 5.11 RFM可视化分析
import matplotlib.pyplot as plt

fig,[ax1,ax2,ax3] = plt.subplots(1,3,figsize=(16,4))

# 绘制条形图
user_features.groupby('Total_Score')['recency'].mean().plot(kind='bar', colormap='Blues_r',ax=ax1, rot=360)
user_features.groupby('Total_Score')['frequency'].mean().plot(kind='bar', colormap='Blues_r',ax=ax2, rot=360)
user_features.groupby('Total_Score')['monetary'].mean().plot(kind='bar', colormap='Blues_r',ax=ax3, rot=360)

# 设置y轴标签
ax1.set_ylabel('recency')
ax2.set_ylabel('frequency')
ax3.set_ylabel('monetary')

# 自动调整子图间距
plt.tight_layout()
plt.show()

In [ ]:
# 5.12 客户价值等级数据的读取
import pandas as pd
import pymysql

# 从数据库读入客户价值等级数据
connection = pymysql.connect(host='172.17.6.26', 
                             port = 3306, 
                             user = 'raa_user', 
                             password = 'bigdata123', 
                             db = 'transactions',
                             charset = 'utf8mb4')
try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM user_potential"
        cursor.execute(sql)
        level = cursor.fetchall() 
finally:
    connection.close()

level=pd.DataFrame(list(level),columns = ['user_id','user_potential'])

print(level.head())

In [ ]:
# 5.13 One-Hot编码
import pandas as pd

# 对消费渠道进行One-hot编码
user_features_predict = pd.get_dummies(data = user_features_predict,columns = ['consumption_channel'],prefix = '',prefix_sep='')

print(user_features_predict.head())

In [ ]:
# 5.14 日期等距离离散化
import pandas as pd

# 对网购首单时间进行离散化
user_features_predict['online_buy_first_date'] = pd.cut(user_features_predict['online_buy_first_date'],bins = 6,labels = False,duplicates = 'drop')

# 对网购最后一次时间进行离散化
user_features_predict['online_buy_last_date'] = pd.cut(user_features_predict['online_buy_last_date'],bins = 6,labels = False,duplicates = 'drop')

print(user_features_predict[['online_buy_first_date','online_buy_last_date']])


In [ ]:
# 5.15 连续型数值等频离散化
import pandas as pd

# 所有数值为连续型的列
continuous_col = ['max_consume_amt', 'consume_order_ratio', 'mon_consume_frq', 'online_cnt', 'online_amt', 'online_avg_amt', 'mon_online_frq','dining_cnt','dining_amt', 'dining_avg_amt', 'business_travel_cnt','business_travel_amt', 'business_travel_avg_amt', 'mon_business_travel_frq', 'car_cnt', 'car_amt', 'phone_fee_cnt', 'phone_fee_amt', 'credit_card_repay_cnt', 'credit_card_repay_amt','cash_advance_cnt', 'cash_advance_amt', 'payroll','total_transactions_amt', 'total_transactions_cnt', 'withdraw_cnt','withdraw_amt', 'total_deposit', 'total_withdraw', 'transfer_cnt','transfer_amt', 'transfer_mean', 'internet_media_cnt', 'internet_media_amt', 'public_pay_amt', 'return_cnt',  'recency', 'frequency', 'monetary']

# 对所有连续型数值的列进行等频离散化
user_features_predict[continuous_col] = user_features_predict[continuous_col].apply(lambda x:pd.qcut(x,6,duplicates = 'drop',labels=False))
    
print(user_features_predict.head())

In [ ]:
# 5.16 训练测试集的划分
import pandas as pd
from sklearn.model_selection import train_test_split

# 提取有价值等级客户的数据
customers = user_features_predict[user_features_predict["user_potential"].notnull()]

# 提取X，y
y = customers.user_potential
X = customers.drop(['user_potential'],axis=1)

# 划分训练集、测试集
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 33)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
# 5.17 构建客户价值等级预测模型
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 设置分类器
clf = LogisticRegression()

# 训练
clf.fit(x_train,y_train)

# 预测
y_predict = clf.predict(x_test)

# 计算准确率
score = clf.score(x_test,y_test)

# 以百分数形式输出并保留两位小数
print('%.2f%%'%(score*100))

# 输出分类报告
report = classification_report(y_test,y_predict)
print(report)


In [ ]:
# 5.18 预测客户价值阶段
import pandas as pd

# 选取价值等级未知的客户数据
null_customers = user_features_predict[user_features_predict['user_potential'].isnull()]

# 对价值等级未知的客户进行预测
null_customers["user_potential"] =  clf.predict(null_customers.drop(['user_potential'],axis = 1))

# 将数据合并在原始表中
user_features['user_potential'] = pd.concat([customers['user_potential'],null_customers['user_potential']],axis=0)

print(user_features["user_potential"].value_counts())

In [ ]:
# 5.19 客户价值等级分布
import pandas as pd
import matplotlib.pyplot as plt

# 计算不同客户价值等级的客户数量
lmt_num = user_features["user_potential"].value_counts()

fig = plt.figure(figsize=(8,6))

# 绘制柱状图，查看客户价值等级取值分布情况
lmt_num.plot(kind = "bar",rot=360,width = 0.15)

# 设置柱形名称
plt.xticks([0,1],['低价值','高价值'])

# 设置x、y轴标签
plt.ylabel("客户数量")
plt.xlabel("客户价值等级")

# 设置标题
plt.title("客户价值等级分布图")

plt.show()

In [ ]:
# 5.20 客户标签间的相关性
import seaborn as sns
import matplotlib.pyplot as plt

# 取所有标签的后15列
last_features = user_features.iloc[:,-15:]

# 计算各标签间的皮尔森相关系数
corr = abs(last_features.corr())

fig = plt.figure(figsize = (10,10))

# 设置颜色风格
cmap = sns.cubehelix_palette(8, start = 2, rot = 0, dark = 0, light =0.95)

# 绘制热力图
sns.heatmap(corr,annot = True,linewidths = 0.05,cmap = cmap)

plt.show()

In [ ]:
# 5.21 客户价值等级与是否拥有高端消费的可视化分析
import pandas as pd
import matplotlib.pyplot as plt

# 计算交叉表
counts = pd.crosstab(user_features["high_consumption"],user_features["user_potential"])

# 重命名索引和列名
counts.index = ["无高端消费","有高端消费"]
counts.columns= ["低价值客户","高价值客户"]

fig = plt.figure(figsize=(8,6))

# 绘制柱状图
counts.plot(kind ="bar",rot = 360,width = 0.3)

# 设置x、y轴标签
plt.ylabel("人数")
plt.xlabel("有无高端消费")

# 设置标题
plt.title("客户价值等级与是否拥有高端消费之间的关系")

plt.show()

In [ ]:
# 5.22 月均消费频度的可视化分析
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 设置画布大小
fig = plt.figure(figsize=(8,6))

# 绘制月均消费频度的取值分布直方图
sns.distplot(user_features['mon_consume_frq'],color='pink')

# 设置标题
plt.title("月均消费频度取值分布")

# 设置x轴标签
plt.xlabel("月均消费频度")

plt.show()

In [ ]:
# 5.23 客户价值等级与月均消费频度的可视化分析
import pandas as pd
import matplotlib.pyplot as plt

# 将月均消费频度取值划分到不同区间
frq_range = pd.cut(user_features['mon_consume_frq'],[0,15,30,45,60,75],right = False)

# 计算交叉表
counts = pd.crosstab(frq_range,user_features['user_potential'])

# 重命名列名
counts.columns = ['低价值客户','高价值用户']

fig = plt.figure(figsize = (8,6))

# 绘制柱状图
counts.plot(kind = 'bar',rot = 360,width = 0.7)

# 设置x、y轴标签
plt.xlabel('月均消费频度')
plt.ylabel('人数')

# 设置标题
plt.title('客户价值等级与月均消费频度的关系')

plt.show()

In [ ]:
# 5.25 合并交易附言
import pandas as pd

## 分组合并交易附言
describe_sum = data.groupby('user_id')['describe_cutted'].sum()

print(describe_sum)

In [ ]:
# 5.26 CountVectorizer词频矩阵计算
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 建立CountVectorizer模型
count_vec = CountVectorizer(stop_words = stop_words_list,min_df = 5,max_df = 5000,max_features = 100)

# 计算词频矩阵
sparse_result_count = count_vec.fit_transform(describe_sum)

# 输出稀疏矩阵
print(sparse_result_count)

# 输出关键词
print('\nvocabulary list:\n\n',count_vec.get_feature_names())

# 输出关键词编号
print('\nvocabulary dic:\n\n',count_vec.vocabulary_)


In [ ]:
# 5.27 TfidfVectorizer词频矩阵计算
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 建立TfidfVectorizer模型
tfidf_vec = TfidfVectorizer(stop_words = stop_words_list,min_df = 5,max_df = 5000,max_features = 100)

# 计算tfidf矩阵
sparse_result_tfidf = tfidf_vec.fit_transform(describe_sum)

# 输出稀疏矩阵
print(sparse_result_tfidf)

# 输出关键词
print('\nvocabulary list:\n\n',tfidf_vec.get_feature_names())

# 输出关键词编号
print( '\nvocabulary dic :\n\n',tfidf_vec.vocabulary_)

# 与user_features表进行合并
user_features = pd.concat([user_features,pd.DataFrame(sparse_result_tfidf.toarray(),index = user_features.index)],axis=1)

# 输出合并后user_features表的行列数
print('\n合并后客户标签表的行列数为:\n\n',user_features.shape)

# 重命名列名
columns_mapping = {value:key for key, value in tfidf_vec.vocabulary_.items()}
user_features.rename(columns = columns_mapping,inplace=True)

# 输出重命名后user_features表的列名
print('\n合并后客户标签表的列名为:\n\n',user_features.columns.values)

In [ ]:
# 5.28 描绘用户画像
from wordcloud import WordCloud         
import matplotlib.pyplot as plt

# 选取文本
describe_selected = describe_sum[user_features['total_transactions_cnt'].idxmax()]

fig = plt.figure(figsize=(10,10))

# 创建词云对象
wordcloud = WordCloud(background_color="white", 
                      mask = background_male,
                      stopwords = stop_words,
                      font_path = "SimHei.ttf",
                      collocations = False,
                      scale = 4,
                      random_state = 30)

# 生成词云
wordcloud.generate(describe_selected)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# 5.29 客户文本标签的分析
import pandas as pd

# 选取存放文本标签词频的行和列
tfidf_features = user_features.loc[10014615][-100:]

# 选取出现最多的前20个交易附言关键词
top_describe = tfidf_features.sort_values(ascending = False)[:20]

print(top_describe)

In [ ]:
# 5.30 客户数值标签的分析
import pandas as pd

# 查看该客户的数值标签
number_feature = user_features.loc[10014615][0:50]

print(number_feature)

In [ ]:
# 5.31 描绘用户画像
from wordcloud import WordCloud         
import matplotlib.pyplot as plt

# 选取文本
describe_selected = describe_sum[12521063]

fig = plt.figure(figsize=(10,10))

# 创建词云对象
wordcloud = WordCloud(background_color="white", 
                      mask = background_female,
                      stopwords = stop_words,
                      font_path = "SimHei.ttf",
                      collocations = False,
                      scale = 4,
                      random_state = 30)

# 生成词云
wordcloud.generate(describe_selected)

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# 5.32 客户文本标签的分析
import pandas as pd

# 选取存放文本标签词频的行和列
tfidf_features = user_features.loc[12521063][-100:]

# 选取出现最多的前20个交易附言关键词
top_describe = tfidf_features.sort_values(ascending = False)[:20]

print(top_describe)

In [ ]:
# 5.33 客户数值标签的分析
import pandas as pd

# 查看该客户的数字特征
number_feature = user_features.loc[12521063][0:50]

print(number_feature)

# 第六章

In [ ]:
# 6.3 时间差的计算
import pandas as pd

# 选取观察点
standard_day=pd.Timestamp(2018,1,1)

# 计算月数
consume['distance']= (standard_day-consume['pay_time'])/pd.Timedelta(days=30)

print(consume.head(2))

In [ ]:
# 6.4
import numpy as np
import pandas as pd

eta = 0.5
text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 筛选彩票类的消费记录
lottery_ticket = consume[consume['describe'].str.contains(text_list)]

# 基于时间的商品兴趣度
user_features['time_penalty'] = lottery_ticket.groupby('user_id')['distance'].agg(lambda x:sum(np.exp(-eta*x)))

print(user_features['time_penalty'].head())

In [ ]:
# 6.5
import numpy as np
import pandas as pd

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 筛选彩票类的消费记录
lottery_ticket = consume[consume['describe'].str.contains(text_list)]

# 计算消费金额
user_features['payment_sum'] = lottery_ticket.groupby('user_id')['payment'].sum()

print(user_features['payment_sum'].head())

In [ ]:
# 6.6 基于tf-idf的商品兴趣度计算
import pandas as pd

# 选取字段
lottery_tfidf = user_features[['停彩','大乐透','双色球','福利彩票','彩票','竞彩','追号']]

# tfidf求和
user_features['tfidf_sum'] = lottery_tfidf.sum(axis = 1)  #按行相加

print(user_features['tfidf_sum'].head())

In [ ]:
# 6.7 数据归一化
import pandas as pd
import numpy as np

# sigmoid归一化
user_features['time_penalty'] = user_features[['time_penalty']].apply(lambda x:1/(1+np.exp(-x)))

# min-max归一化
user_features['payment_sum'] = user_features[['payment_sum']].apply(lambda x : (x - x.min())/(x.max()-x.min()))
user_features['tfidf_sum'] = user_features[['tfidf_sum']].apply(lambda x : (x - x.min())/(x.max()-x.min()))

print(user_features[['time_penalty','payment_sum','tfidf_sum']].head(10))

In [ ]:
# 6.8 商品兴趣度排行榜的综合计算
import pandas as pd

# 计算final_score
user_features['final_score'] = user_features['tfidf_sum'] + user_features['time_penalty'] + user_features['payment_sum']

# top10
top10 = user_features.sort_values('final_score',ascending=False).head(10)

print(top10[['time_penalty','payment_sum','tfidf_sum','final_score']])

In [ ]:
# 6.9 交易次数可视化
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 选择排名前3的user_id
top3 = top10.index.values[:3]

# 不同ID的客户用不同的线段表示
styles=['--','-',":"]

fig = plt.figure(figsize=(15,6))

# 绘制折线图
for user_id,style in zip(top3,styles):
     # 筛选用户的交易记录
    top_select = consume.loc[consume['user_id'] == user_id]
     # 匹配文本关键词
    top_select = top_select[top_select['describe'].str.contains(text_list)]
     # 按天分组，计算交易次数并绘图
    top_select.groupby(top_select['pay_time'].dt.date).size().plot(style=style)

# 设置标题
plt.title('彩票交易次数变化')

# 设置x,y轴标签
plt.xlabel('日期')
plt.ylabel('交易次数')
# 设置图例
plt.legend(labels=['第1名','第2名','第3名'])

# 设置日期显示格式
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.show()

In [ ]:
# 6.10 交易金额的可视化
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 选择排名前3的user_id
top3 = top10.index.values[:3]

# 不同ID的客户用不同的线段表示
styles = ['--','-',":"]

fig = plt.figure(figsize=(15,6))

for user_id,style in zip(top3,styles):
    # 筛选用户的交易记录
    top_select = consume.loc[consume['user_id'] == user_id]
    # 匹配文本关键词
    top_select = top_select[top_select['describe'].str.contains(text_list)]
    # 按天分组，计算交易金额并绘图
    top_select.groupby(top_select['pay_time'].dt.date)['payment'].sum().plot(style = style)

# 设置标题
plt.title('彩票交易金额变化')

# 设置x,y轴标签
plt.xlabel('日期')
plt.ylabel('消费金额')

# 设置图例
plt.legend(labels = ['第1名','第2名','第3名'])

# 设置日期显示格式
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.show()

In [ ]:
# 6.11 目标客户的筛选
import pymysql
import pandas as pd

# 构建连接对象
connection = pymysql.connect(host='172.17.6.26', 
                             port=3306, 
                             user='raa_user', 
                             password='bigdata123', 
                             db='transactions',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM user_features \
        WHERE business_travel_cnt > 0 AND credit_card_repay_cnt > 0 \
        AND is_installment= 1 AND sleep_customers = 0\
        ORDER BY credit_card_repay_cnt DESC LIMIT 200"
        cursor.execute(sql)
        collected_users = cursor.fetchall()

finally:
    connection.close()

# 查看筛选出的客户信息
collected_users = pd.DataFrame(collected_users)
print(collected_users[['user_id','business_travel_cnt','credit_card_repay_cnt']])